# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Preprocessing Data- Label Encoding


In [5]:
# Check for non numerical columns.
pd.set_option('display.max_rows', 100)
t = df.columns.to_series().groupby(df.dtypes).groups
t

{dtype('float64'): Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
        'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
        'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
        'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
        'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
        'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
        'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
        'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
        'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
        'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
        'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
        'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
        'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
        'mort_acc', 'mths_since_r

In [6]:
# Print out unique values for each object column.

print('home_ownership')
print(df['home_ownership'].unique())
print('----------------------')
print('verification_status')
print(df['verification_status'].unique())
print('----------------------')
print('issue_d')
print(df['issue_d'].unique())
print('----------------------')
print('loan_status')
print(df['loan_status'].unique())
print('----------------------')
print('pymnt_plan')
print(df['pymnt_plan'].unique())
print('----------------------')
print('initial_list_status')
print(df['initial_list_status'].unique())
print('----------------------')
print('next_pymnt_d')
print(df['next_pymnt_d'].unique())
print('----------------------')
print( 'application_type')
print(df[ 'application_type'].unique())
print('----------------------')
print('hardship_flag')
print(df['hardship_flag'].unique())
print('----------------------')
print('debt_settlement_flag')
print(df['debt_settlement_flag'].unique())
print('----------------------')

home_ownership
['RENT' 'MORTGAGE' 'OWN' 'ANY']
----------------------
verification_status
['Source Verified' 'Verified' 'Not Verified']
----------------------
issue_d
['Mar-2019' 'Feb-2019' 'Jan-2019']
----------------------
loan_status
['low_risk' 'high_risk']
----------------------
pymnt_plan
['n']
----------------------
initial_list_status
['w' 'f']
----------------------
next_pymnt_d
['May-2019' 'Apr-2019']
----------------------
application_type
['Individual' 'Joint App']
----------------------
hardship_flag
['N']
----------------------
debt_settlement_flag
['N']
----------------------


In [7]:
# Make a list of columns to be encoded.

#Using LabelEncoder:

columns_toEncode=[ 
        'pymnt_plan', 'next_pymnt_d', 
        'hardship_flag', 'debt_settlement_flag']

# Using binary Encoding:

Binary_Encode=['application_type','initial_list_status',
               'home_ownership', 'verification_status']

# Custome Encoding:df['issue_d']
ce={'Mar-2019':3, 'Feb-2019':2, 'Jan-2019':1}


In [8]:
# Import LabelEncoder for converting the datatype from object to to numeric.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in columns_toEncode:
    
    df[col] = le.fit_transform(df[col])

# Binary Encoding:
df=pd.get_dummies(df,columns=Binary_Encode)

#Custom Encoding:
df["issue_d"] = df["issue_d"].apply(lambda x: ce[x])
    
df.head(5)

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,application_type_Joint App,initial_list_status_f,initial_list_status_w,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,3,low_risk,0,27.24,0.0,0.0,...,0,0,1,0,0,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,3,low_risk,0,20.23,0.0,0.0,...,0,0,1,0,1,0,0,0,0,1
2,20000.0,0.2000,529.88,56000.0,3,low_risk,0,24.26,0.0,0.0,...,0,0,1,0,1,0,0,0,0,1
3,10000.0,0.1640,353.55,92000.0,3,low_risk,0,31.44,0.0,1.0,...,0,0,1,0,0,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,3,low_risk,0,18.76,0.0,1.0,...,0,0,1,0,1,0,0,1,0,0


In [9]:
df.shape

(68817, 93)

# Split the Data into Training and Testing

In [11]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df['loan_status']

In [12]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=1)

## Preprocessing Data- Scaling


In [13]:
# Import Scaler
from sklearn.preprocessing import StandardScaler

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled=X_scaler.transform(X_train)
X_test_scaled=X_scaler.transform(X_test)



In [16]:
# Check to make sure the coulmns are scaled.
import numpy as np
print(np.mean(X_train_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

1.6079863507862455e-16
1.0
-0.013206556680231171
0.9943407123136578


In [17]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [18]:
#Check shape.
X_train.shape,y_train.shape

((51612, 92), (51612,))

In [19]:
# Check counter for y train.
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [21]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=1)
X_resampled,y_resampled=ros.fit_resample(X_train_scaled,y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [22]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test_scaled)

from sklearn.metrics import balanced_accuracy_score
bas=balanced_accuracy_score(y_test,y_pred)
bas


0.8384035579657123

In [24]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   84,    17],
       [ 2649, 14455]], dtype=int64)

In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.83      0.85      0.06      0.84      0.70       101
   low_risk       1.00      0.85      0.83      0.92      0.84      0.70     17104

avg / total       0.99      0.85      0.83      0.91      0.84      0.70     17205



### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train_scaled, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [27]:
# Train the Logistic Regression model using the resampled data
model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test_scaled)

bas=balanced_accuracy_score(y_test,y_pred)
bas


0.8383248316646446

In [29]:
# Display the confusion matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   82,    19],
       [ 2313, 14791]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.81      0.86      0.07      0.84      0.70       101
   low_risk       1.00      0.86      0.81      0.93      0.84      0.71     17104

avg / total       0.99      0.86      0.81      0.92      0.84      0.71     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc=ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [32]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test_scaled)

bas=balanced_accuracy_score(y_test,y_pred)
bas

0.8039289634061628

In [34]:
# Display the confusion matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   86,    15],
       [ 4167, 12937]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.85      0.76      0.04      0.80      0.65       101
   low_risk       1.00      0.76      0.85      0.86      0.80      0.64     17104

avg / total       0.99      0.76      0.85      0.86      0.80      0.64     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'high_risk': 51366, 'low_risk': 47362})

In [37]:
# Train the Logistic Regression model using the resampled data
model=LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [38]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test_scaled)

bas=balanced_accuracy_score(y_test,y_pred)
bas

0.8379257008956273

In [39]:
# Display the confusion matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   83,    18],
       [ 2496, 14608]], dtype=int64)

In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.82      0.85      0.06      0.84      0.70       101
   low_risk       1.00      0.85      0.82      0.92      0.84      0.70     17104

avg / total       0.99      0.85      0.82      0.92      0.84      0.70     17205

